In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


In [4]:
base_dir = r"c:\Users\polaa\OneDrive\Desktop\plant-disease-classification\data\processed"
train_dir = os.path.join(base_dir, "train")
val_dir   = os.path.join(base_dir, "val")
test_dir  = os.path.join(base_dir, "test")

img_size = (224, 224)
batch_size = 32



In [5]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"
)

val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"
)

test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)

num_classes = train_gen.num_classes
num_classes


Found 507 images belonging to 9 classes.
Found 142 images belonging to 9 classes.
Found 82 images belonging to 9 classes.


9

In [6]:
base_model = ResNet50(
    include_top=False,
    weights="imagenet",
    input_shape=(img_size[0], img_size[1], 3)
)

# تجميد كل طبقات ResNet50 في الأول
for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = models.Model(inputs=base_model.input, outputs=outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 29s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 24,114,569 (91.99 MB)

 Trainable params: 526,857 (2.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [7]:
checkpoint = ModelCheckpoint(
    "models/resnet50_phase1_best.h5",
    monitor="val_accuracy",
    save_best_only=True,
    mode="max",
    verbose=1
)

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.3,
    patience=3,
    verbose=1
)

history_phase1 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=15,
    callbacks=[checkpoint, early_stop, reduce_lr]
)


Epoch 1/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 920ms/step - accuracy: 0.2768 - loss: 2.1401
Epoch 1: val_accuracy improved from None to 0.87324, saving model to models/resnet50_phase1_best.h5


16/16 ━━━━━━━━━━━━━━━━━━━━ 30s 1s/step - accuracy: 0.4556 - loss: 1.5677 - val_accuracy: 0.8732 - val_loss: 0.4913 - learning_rate: 0.0010
Epoch 2/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 798ms/step - accuracy: 0.7829 - loss: 0.5803
Epoch 2: val_accuracy improved from 0.87324 to 0.93662, saving model to models/resnet50_phase1_best.h5


16/16 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.8067 - loss: 0.5475 - val_accuracy: 0.9366 - val_loss: 0.2745 - learning_rate: 0.0010
Epoch 3/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 820ms/step - accuracy: 0.9177 - loss: 0.2844
Epoch 3: val_accuracy did not improve from 0.93662
16/16 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.9053 - loss: 0.3148 - val_accuracy: 0.9366 - val_loss: 0.1911 - learning_rate: 0.0010
Epoch 4/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 769ms/step - accuracy: 0.9176 - loss: 0.2936
Epoch 4: val_accuracy improved from 0.93662 to 0.96479, saving model to models/resnet50_phase1_best.h5


16/16 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.9250 - loss: 0.2491 - val_accuracy: 0.9648 - val_loss: 0.1392 - learning_rate: 0.0010
Epoch 5/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 781ms/step - accuracy: 0.9614 - loss: 0.1378
Epoch 5: val_accuracy improved from 0.96479 to 0.98592, saving model to models/resnet50_phase1_best.h5


16/16 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.9606 - loss: 0.1507 - val_accuracy: 0.9859 - val_loss: 0.0960 - learning_rate: 0.0010
Epoch 6/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 775ms/step - accuracy: 0.9590 - loss: 0.1295
Epoch 6: val_accuracy did not improve from 0.98592
16/16 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.9645 - loss: 0.1245 - val_accuracy: 0.9577 - val_loss: 0.1295 - learning_rate: 0.0010
Epoch 7/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 871ms/step - accuracy: 0.9791 - loss: 0.0958
Epoch 7: val_accuracy did not improve from 0.98592
16/16 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.9684 - loss: 0.1048 - val_accuracy: 0.9718 - val_loss: 0.0830 - learning_rate: 0.0010
Epoch 8/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 873ms/step - accuracy: 0.9766 - loss: 0.0918
Epoch 8: val_accuracy did not improve from 0.98592
16/16 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.9803 - loss: 0.0764 - val_accuracy: 0.9859 - val_loss: 0.0720 - learning_rate: 0.0010
Epoch 9/15
16/16 ━━━━━━━━━━━━━━━━

16/16 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - accuracy: 0.9901 - loss: 0.0416 - val_accuracy: 0.9930 - val_loss: 0.0548 - learning_rate: 0.0010
Epoch 13/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 827ms/step - accuracy: 0.9928 - loss: 0.0376
Epoch 13: val_accuracy did not improve from 0.99296
16/16 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - accuracy: 0.9941 - loss: 0.0362 - val_accuracy: 0.9789 - val_loss: 0.0532 - learning_rate: 0.0010
Epoch 14/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 877ms/step - accuracy: 0.9985 - loss: 0.0264
Epoch 14: val_accuracy did not improve from 0.99296
16/16 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.9941 - loss: 0.0314 - val_accuracy: 0.9859 - val_loss: 0.0549 - learning_rate: 0.0010
Epoch 15/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 813ms/step - accuracy: 0.9997 - loss: 0.0293
Epoch 15: val_accuracy did not improve from 0.99296
16/16 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.9980 - loss: 0.0288 - val_accuracy: 0.9859 - val_loss: 0.0439 - learning_rate: 0.0010


In [8]:
for layer in base_model.layers[-30:]:
    layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

history_phase2 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=2,
    callbacks=[early_stop, reduce_lr]
)


Epoch 1/2
16/16 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.9250 - loss: 0.2201 - val_accuracy: 0.8803 - val_loss: 0.4633 - learning_rate: 1.0000e-04
Epoch 2/2
16/16 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.9961 - loss: 0.0252 - val_accuracy: 0.9507 - val_loss: 0.1507 - learning_rate: 1.0000e-04


In [9]:
test_loss, test_acc = model.evaluate(test_gen)
print("Test Accuracy:", test_acc)


3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 445ms/step - accuracy: 0.9146 - loss: 0.1908
Test Accuracy: 0.9146341681480408


In [10]:
model.save("resnet50_final.h5")
print("Model saved as resnet50_final.h5")


Model saved as resnet50_final.h5
